## - 데이터 분석 & 데이터 시각화

## 2. 데이터 시각화
데이터 시각화란 데이터 분석의 결과를 사람이 쉽게 이해할 수 있도록 도표등의 수단을 통해 정보를 명확하고 효과적으로 전달하는 것을 말합니다.



In [ ]:
# 런타임 접속
# (1)미리 조교가 준비해 둔 데이터파일을 다운받습니다. house_price.zip이라는 파일이 다운받아집니다.
!wget --no-check-certificate 'https://docs.google.com/uc?export=download&id=1UTQ6r2-_OGOF6VMx7WK4jgQoU7cTeuMe' -O house_price.zip

In [ ]:
# 압축을 해제하고, house_price라는 폴더에 저장합니다.
!unzip house_price.zip -d house_price

In [ ]:
#Data 설명을 매번 읽어오는일은 번거로운 일이다. dictionary형태로 저장하고 원할때마다 정보를 읽어오는 코드를 작성해보자
with open('house_price/data_description.txt', 'r') as f:
    alldata = f.readlines()
    f.close()

help_data = dict()
title = ''
des   = ''
for line in alldata:
    if line[0].isalpha() or line[0].isdigit():
        if title != '':
            if title == "Bedroom":
                title = "BedroomAbvGr"
            if title == "Kitchen":
                title = "KitchenAbvGr"
            help_data[title.lower()] = des
            des = ""
        
        title = line.split(':')[0]
    des += line

help_data[title.lower()] = des
help_data["saleprice"] = "SalePrice: the property's sale price in dollars. This is the target variable that you're trying to predict."

def show_data(key):
    if key.lower() in help_data.keys():
        print(help_data[key.lower()])
    else:
        print("오타가 있습니다! 존재하지 않는 항목을 입력하셨습니다.")

In [ ]:
##show_data실험
show_data('mssubclass')

In [ ]:
# 시각화에 필요한 라이브러리를 불러옵니다.
## 다른 프로그램의 구성 요소를 사용하기 위해 미리 만들어 둔 프로그램 조각 
import numpy as np # linear algebra....
import pandas as pd 
%matplotlib inline
from matplotlib import pyplot as plt #Visualization of the data....
from mpl_toolkits import mplot3d
import warnings
import seaborn as sns
warnings.filterwarnings("ignore")
pd.set_option("display.max_columns",None)
pd.set_option("display.max_rows",None)

In [ ]:
## os module에서 파일 및 디렉토리를 탐색하는 방법
import os
os.listdir('house_price')

In [ ]:
#파일을 불러오고 파일의 일부를 보여줍니다.(Method)
df_train=pd.read_csv("house_price/train.csv")

In [ ]:
# 데이터엔 어떤 항목이 있는지 볼까요?
print("데이터 항목 리스트:\n",df_train.columns)

In [ ]:
df_train.head() ## 개수, tail, 특정 column만

In [ ]:
df_train.describe()

In [ ]:
# 데이터 개수가 몇개인지, 데이터 속성의 개수가 몇개인지 등 데이터가 어떻게 생겼는지 볼까요?
print(len(df_train),len(df_train.columns))
print("(데이터 개수,데이터의 Feature의 개수):",df_train.shape)

In [ ]:
# 데이터의 대략적인 구조를 보여줍니다.
df_train.info()

In [ ]:
# 데이터에는 항상 결측값이 존재합니다. 데이터가 없는 부분을 시각화해봅시다!
import missingno as mn
mn.matrix(df_train,color=(0,0,0))

In [ ]:
import seaborn as sns

In [ ]:
sns.set(rc = {'figure.figsize':(10,10)})
sns.heatmap(df_train.isnull(),cbar=False)

In [ ]:
# 전체 heatmap 확인 : 
sns.heatmap(df_train.corr())
# 특정 항복 GarageCars와 연관성 찾기
# sns.heatmap(df_train.corr()[['SalePrice']],annot=False)

In [ ]:
# 다운받았던 데이터의 항목별 결손 비율이 어느정도인지 확인합니다.
null_list = []
for features in df_train.columns:
  if df_train[features].isnull().sum()>=1:
    null_list.append(features)

for i in null_list:
  print(i,round(df_train[i].isnull().mean(),4),'% missing values')

In [ ]:
# 다운받았던 데이터의 항목별 결손 비율이 어느정도인지 확인합니다.(ListComprehension) 코드 간결화
features_with_na=[features for features in df_train.columns if df_train[features].isnull().sum()>=1]
for feature in features_with_na:
    print(feature, np.round(df_train[feature].isnull().mean(), 4),  ' % missing values')

In [ ]:
# 결측이 존재하는 항목을 모두 제거해줍시다. (결측 데이터는 추후 에러의 원인이 될 수 있습니다.)
# 제거 방식 예시를 통해서 보여주기
df_train = df_train.dropna(axis=1)
## 그 axis=1을 통해 열값을 삭제 (예시 보여주기)
# 제거가 잘되었는지 확인해볼까요?
df_train.info()
# 기존 80-> 62개로 줄었습니다!

In [ ]:
## 추후 에러의 원인이 되는 결측값(Null)값 제거와 DataFrame 분석을 완료! 세부적인 분석을 시작(수치형 데이터와 범주형 데이터 설명하기)
# 수치형 항목을 뽑아내봅시다.
numerical_features = [feature for feature in df_train.columns if df_train[feature].dtypes != 'O']
# 수치형 항목의 개수는 35개네요.
print('Number of numerical variables: ', len(numerical_features))

In [ ]:
# 어떤 수치형 항목들이 있는지 표로 확인해봅시다.
df_train[numerical_features].head().style.set_properties(**{"background-color": "#98FB98","color": "black", "border-color": "black"})

In [ ]:
# 수치형 항목중, 특히 제가 궁금한 정보 연도와 관련된 항목을 뽑아봅시다.
year_feature = [feature for feature in numerical_features if 'Yr' in feature or 'Year' in feature]
# 어떤 항목이 있을까요?
year_feature

In [ ]:
# 각각이 어떤 값인지 설명을 봐봅시다.
for f in year_feature:
    show_data(f)

In [ ]:
# 각 연도 항목에는 어떤 데이터들이 존재할까요?
for feature in year_feature:
    print(feature, sorted(df_train[feature].unique()),end='\n\n')

In [ ]:
df_train.groupby('YrSold')['SalePrice'].median().plot()

In [ ]:
# 특정 항목(yrSold,yearRemodAdd,YrSold)과, 집값(SalePrice)과의 연관성을 그래프로 확인해봅시다!
# 적절한 값을 통해, 도표의 크기(가로, 세로)를 정할 수 있습니다.  / 공식 홈페이지 확인
plt.figure(figsize=(8,8))
# Groupby함수를 통해 YrSold(Year Sold) 항목과, 판매가격('SalePrice') 두가지의 연관관계를 나타내 봅시다.
df_train.groupby('YrSold')['SalePrice'].median().plot(color = "orange",linestyle = "-",linewidth=3,marker='v')
## color,linestyle,linewidth,marker
# 원하신다면, 'YrSold' 부분과 'SalePrice' 부분을 다른항목으로 변경하여 관계성을 확인할 수 있습니다.
plt.xlabel('Year Sold',fontsize = 15)
# # x축에 Year Sold라고 표시합니다.
plt.ylabel('Median House Price',fontsize =15)
# # y축에 Median House Price라고 표시합니다.
plt.title("House Price vs YearSold",fontsize=22)
# 도표의 제목을 짓습니다.
plt.show()

In [ ]:
## Seaborn histplot으로 대략적인 그림 보이기 [data,x,y] 
sns.histplot(data=df_train,x='YrSold',y='SalePrice')

# Discrete Feature

# 연속적이지 않은 값과 집값과의 관계 분석을 해봅시다.(화장실 갯수, 욕조 갯수 등등)


In [ ]:
# 본 Dataset에서 화장실의 개수와 같은 Discrete Feature는 1, 2, 3, 4, 5 등으로 값이 딱 떨어지므로, 중복을 제외했을 때 unique 값이 총 25를 넘지 않습니다. 
# (예를 들어, 대부분의 집은 화장실의 개수가 대부분 1,2,3,4개 정도의 4가지 경우의 수로 끝나지, 25가지 이상의 경우의수가 존재하지 않습니다.)
# 따라서, unique한 값의 개수가 25 미만인 값들을 discrete로 처리합니다.
# (반면에, continuous (연속적 값), 예를 들어 집의 평수, 수영장의 넓이, 마당의 넓이 등은 1,2,3처럼 딱 떨어지지않고, 1.1 1.2 1.3과 같이 무수히 많은 unique 값을 가지겠죠?)
discrete_feature=[feature for feature in numerical_features if len(df_train[feature].unique())<25 and feature not in year_feature+['Id']]
print("Discrete Variables Count: {}".format(len(discrete_feature)))

In [ ]:
# 각각이 어떤 값인지 설명을 봐봅시다.
for f in discrete_feature:
    show_data(f)

미국의 주택문화는 아파트에 주로 사는 우리 나라와 달라서, 지상층 / 지하층 유무를 굉장히 중요시 여깁니다.  
BsmtFullBath항목 : 지하층에 있는 Full Bath의 개수  
(FullBath : 샤워시설, 변기가 함께 존재), (HalfBath : 샤워시설 없이 화장실만)  
FullBath항목  : 지상층에 있는 Full Bath의 개수  
뒤에 AbvGr, AbvGrd가 붙은 데이터는 Above Grade라는 뜻으로, 지상층이라고 생각하시면 됩니다.

# 지상에 위치한 FullBath와 집값의 연관관계를 확인해봅시다!

In [ ]:
#먼저 데이터를 복사해옵니다. (원치 않는 데이터 조작을 방지하기 위함)
data=df_train.copy()

# 차트의 색깔을 정할 수 있습니다. 
# https://htmlcolorcodes.com/ 
# 원하는 컬러의 차트를 그려보세요!
colors = ["#EC4A4A","#2B41F7"]
#기본값, 빨강, 초록
plt.figure(figsize = (6, 6))
data.groupby('FullBath')['SalePrice'].median().plot.bar(color=colors,edgecolor = "black",linewidth=1.5)
# Full Bath 데이터항목과 SalePrice와의 연관관계를 차트로 나타냅니다.
# mean이 아닌 median()을 쓰는 이유는 중앙값이 가장 경향성을 잘 보여주는 지표이기 때문입니다.
'''
    수업 내용
    mean(통계의 함정) : 미국에서 가장 평균 연봉이 높은 학과는 어디일까요? 
    법학과, 컴퓨터학과, 전자과, 의예과 등등을 생각하시겠지만, 사실 의외로 지리학과가 평균 연봉 1억2천만원으로 가장 높게 산정되었습니다.
    그 이유는 뭘까요? 정말 지리학과의 모든 사람들이 그만큼 돈을 벌까요?
    이유는 굉장히 간단합니다. 지리학과 졸업생중 한명이 전설적인 농구선수 마이클 조던이기 때문입니다.
    마이클 조던의 연봉으로 인해, 지리학과 전체 평균이 어마어마하게 높아진 것이죠.
    이런걸 방지하기위해서, 항상 데이터간의 경향성을 살피기 위해서 우리는 평균이 아니라 median(중앙값) 혹은 최빈값을 사용합니다.
'''

plt.xlabel('FullBath',fontsize=15)
plt.ylabel('SalePrice',fontsize =15)
plt.title('FullBath' +" Vs SalePrice",fontsize = 22)
plt.show()

# 총 방의 개수와 집값의 연관관계를 확인해봅시다!

In [ ]:
data=df_train.copy()
colors = ["#FF6347","#98FB98"]
target = 'BedroomAbvGr' # 매번 Feature이름을 바꾸는것은 매우 귀찮은 일입니다. 코드 재사용성을 높이기 위해서, Target이라는 변수에 연관관계를 확인하고싶은 항목을 넣어 사용해봅시다.
plt.figure(figsize = (6, 6))
data.groupby(target)['SalePrice'].median().plot.bar(color=colors,edgecolor = "black",linewidth=1.5)
plt.xlabel(target,fontsize=15)
plt.ylabel('SalePrice',fontsize =15)
plt.title(target +" Vs SalePrice",fontsize = 22)
plt.show()

# [TO-DO] 실습 #1  
그 외에 어떤 Discrete 데이터가 집값과 연관이 있을지 생각해보고, 코드를 작성하여 직접 확인해봅시다!

In [ ]:
discrete_feature

In [ ]:
#sns.histplot(data=df_train,x='HalfBath',y='FullBath')
sns.boxplot(data=df_train,x='HalfBath',y='FullBath')

In [ ]:
#data.groupby(target)['SalePrice'].median().plot.bar(color=colors,edgecolor = "black",linewidth=1.5)
# 정렬시켜서도 확인해보기 -> 아래 코드
data.groupby(target).mean()['SalePrice'].sort_values().plot.bar()

# 함수화
거의 구조가 똑같은 코드를 매번 반복적으로 입력하는것은 매우 비효율적입니다.  
Discrete Data와 가격 연관관계 그래프를 그리는 함수를 정의하고, 편하게 재사용해봅시다.

In [ ]:
# 함수를 정의합니다. 
# target (연관관계 분석을 하고자 하는 항목)을 함수의 매개변수로 사용하여, 그래프를 그리는 함수입니다.
def draw_graph_discrete(target,x,y):
  data=df_train.copy()
  colors = ["#FF6347","#98FB98"]
  plt.figure(figsize = (x, y))
  data.groupby(target)['SalePrice'].median().plot.bar(color=colors,edgecolor = "black",linewidth=1.5)
  plt.xlabel(target,fontsize=15)
  plt.ylabel('SalePrice',fontsize =15)
  plt.title(target +" Vs SalePrice",fontsize = 22)
  plt.show()

In [ ]:
# 매우 간단하고 편하게 사용할 수 있습니다.
draw_graph_discrete('OverallCond',5,5)
## figsize x,y값도 매개변수로 받아보기

In [ ]:
def draw_sns(target,x):
  if x=='hist':
    sns.histplot(data=df_train,x=target,y='SalePrice')
  else:
    sns.boxplot(data=df_train,x=target,y='SalePrice')
draw_sns('FullBath','hist')

# [TO-DO] 실습 #2
Discrete 데이터와 집값의 연관관계를 나타내는 함수를 정의했습니다.  
이를 활용하여 다시 한번 연관관계를 분석해봅시다.

In [ ]:
#====================TODO======================
#  코드 작성 하는 곳 
#  위에 있던 함수를 적절히 활용해서 직접 그래프를 띄워봅시다.
#==============================================

# Continuous Feature (집의 넓이, 마당 평수와 같은 연속적인 값) 

In [ ]:
# 전체 Feature 중에서, Numerical Feature면서, Discrete_Feature에 속하지 않는 데이터들을 추출합니다.
continuous_feature=[feature for feature in numerical_features if feature not in discrete_feature+year_feature+['Id']]
print("Continuous feature Count {}".format(len(continuous_feature)))
# 총 개수는 14개네요!

In [ ]:
# 어떤 데이터가 있는지 살펴봅시다.
# 각각의 의미를 위의 데이터 설명표를 보고 확인해보세요!
# Bsmt는 Basement로 AbvGr와 반대의 의미, 즉 지하를 의미합니다.
# LotArea는 마당을 포함한 집 부지 전체의 넓이를 의미합니다.
# 1stFlrSF는 1층의 평수
# 2stFlrSF는 2층의 평수입니다.
continuous_feature

In [ ]:
for f in continuous_feature:
  print(f)
  show_data(f)

# 데이터의 분포를 살펴봅시다!

In [ ]:
data=df_train.copy()
target = '1stFlrSF'
plt.figure(figsize = (7, 7))
data[target].hist(bins=30,color = "purple",edgecolor = "black", linewidth = 2) 
# bins의 숫자조절을 통해 막대 그래프의 개수를 조절할 수 있음
plt.xlabel(target, fontsize=15)
plt.ylabel("Count", fontsize=15) 
plt.title(target, fontsize = 22) 
plt.grid(color = "palegreen")
plt.show()

In [ ]:
##seaborn library histplot그리기 [data,x]
sns.histplot(data=df_train,x='1stFlrSF')

# 반복문 활용
모든 데이터항목에 대해서 분포를 보고싶습니다.  
위와같은 코드에서 Feature를 바꿔가며 하나하나 출력하는 것은 너무 비효율적입니다.  
for문을 사용하여 한번에 모든 그래프를 출력해봅시다.

In [ ]:
data=df_train.copy()
for target in continuous_feature:
    plt.figure(figsize = (8, 8))
    data[target].hist(bins=25,color = "tomato",edgecolor = "black", linewidth = 1.75) 
    plt.xlabel(target, fontsize=15)
    plt.ylabel("Count", fontsize=15) 
    plt.title(target, fontsize = 22) 
    plt.grid(color = "palegreen")
    plt.show()

# Feature Scaling
Feature의 분포는 보통 매우 다양합니다.  
예를 들어, 누군가는 5평짜리 집에 살 수도 있지만, 누군가는 1000평짜리 집에서 살기도 합니다.  
위 그래프만 보더라도, 매우 소수이지만, LotArea가 200000에 달하는 데이터가 존재합니다.
대부분의 데이터는 앞쪽에 편향되어 있는데 말이죠.  

이러한 데이터의 특성은, Feature간의 연관관계 분석에 큰 악영향을 미칩니다.  
최빈값, 중앙값, 평균값과의 차이가 너무 크기 때문인데요. 

이러한 현상을 방지하기 위해서, 빅데이터쪽에서는 Feature Scaling이라는 방법을 사용합니다.
Feature Scaling의 방법은 매우 다양합니다. 

첫째로, Min-Max Normalization(최소-최대 정규화)가 있습니다.  
<p align="center">
$ x_{scaled} = \frac{x - min}{max - min} $
</p>


둘째로는, Z-Score Normalization이 있습니다. 
<p align="center"> 
$ x_{z-scaled} = \frac{x - \mu}{\sigma}$
</p>

셋째로는, 단순히 log를 활용하여, 데이터 값을 최대한 압축시키는 방식입니다.  
<p align="center">
$ x_{logarithm} = \log(x)$
</p>
위와 같은 방법으로 데이터를 적절히 정규화해줍니다. 

# Continuous Data 연관관계 분석

In [ ]:
sns.histplot(data=df_train,x='FullBath')

In [ ]:
#plt.bar(data=df_train,x='FullBath',height='SalePrice')

In [ ]:
# Feature Scaling을 거친 연관관계를 분석해봅시다.
data=df_train.copy()
target = 'WoodDeckSF'

data[target]=np.log(data[target])
# log를 취하는 이유가 뭘까요?
data['SalePrice']=np.log(data['SalePrice'])
# 3차원 그래프를 그려봅시다!
fig = plt.figure(figsize = (15, 12))
ax = plt.axes(projection ="3d")
ax.scatter3D(data[target],data['SalePrice'],color = "tomato")
plt.xlabel(target,fontsize=15)
plt.ylabel('SalesPrice',fontsize = 15)
plt.title(target + " Vs SalePrice",fontsize=22)
plt.grid(color = "palegreen")
plt.show()

# [TO-DO] 실습 #3
아까 함수 만드는 방법을 배웠습니다.  
Continuous Data도 함수로 만들어보고,  
연관관계를 다양하게 분석해봅시다.

In [ ]:
def draw_graph_continuous(target):
  # 연관관계를 분석해봅시다.
  data=df_train.copy()
  data[target]=np.log(data[target])
  # log를 취하는 이유가 뭘까요?
  data['SalePrice']=np.log(data['SalePrice'])
  # 3차원 그래프를 그려봅시다!
  fig = plt.figure(figsize = (8, 8))
  ax = plt.axes(projection ="3d")
  ax.scatter3D(data[target],data['SalePrice'],color = "tomato")
  plt.xlabel(target,fontsize=15)
  plt.ylabel('SalesPrice',fontsize = 15)
  plt.title(target + " Vs SalePrice",fontsize=22)
  plt.grid(color = "palegreen")
  plt.show()

In [ ]:
draw_graph_continuous('WoodDeckSF')

# Categorical Feature (범주형 자료)
# Ex) 지면 높이 (지하, 언덕, 지상, 푹 꺼진 땅) 등등

In [ ]:
# dtype이 Object인 Feature들을 모두 추출(범주형 데이터)
categorical_features=[feature for feature in df_train.columns if data[feature].dtypes=='O']
# 어떤 범주형 데이터가 있는지 볼까요?
len(categorical_features)
# 각 범주형 데이터가 어떤 값을 지니는지, 각각의 데이터가 뭘 의미하는지 ~~를 보고 참고해봅시다.

In [ ]:
categorical_features

In [ ]:
# 메타데이터를 쉽게 볼 수 있도록 미리 함수로 작성해두었습니다.
# 대소문자 구분 안하셔도 됩니다!
# 각 항목의 데이터가 뭘 의미하는지 확인해보세요!
#show_data('SaleCondition')

# 혹은 아래와 같이, for문으로 전체를 출력할 수도 있습니다. (내용이 길 수 있습니다.)
for f in categorical_features:
  show_data(f)

In [ ]:
df_train[categorical_features].head().style.set_properties(**{"background-color": "#98FB98","color": "black", "border-color": "black"})

In [ ]:
data=df_train.copy()
target = 'Neighborhood' # 중앙 냉난방 시스템 여부
colors = ["#FF6347","#98FB98"]
plt.figure(figsize=(10,10))
data.groupby(target)['SalePrice'].median().plot.bar(color = colors,edgecolor = "black",linewidth = 1.5)
plt.xlabel(target, fontsize = 15)

plt.xticks(rotation=45)
## xticks(rotation)의 각도를 조절해 x축 항목의 각도를 변환시킬 수 있습니다.
plt.ylabel('SalePrice',fontsize = 15)
plt.title(target + " Vs SalePrice ",fontsize = 22)
plt.show()

In [ ]:
# jointplot, histplot, boxplot그려보기
sns.jointplot(data=df_train,x='Street',y='SalePrice')

In [ ]:
# jointplot, histplot, boxplot그려보기
sns.histplot(data=df_train,x='Street',y='SalePrice')

In [ ]:
categorical_features

# [TO-DO] 실습 #4
함수화 시켜보고, Categorical Data와 가격관의 연관관계를 분석해보세요.

In [ ]:
# 직접 Seaborn 홈페이지를 들어가 어떠한 시각화 함수가 있는지 확인해보고 직접 그려봅시다.
sns.catplot(x="RoofStyle", y="SalePrice", data=df_train)

In [ ]:
sns.jointplot(x="RoofStyle", y="SalePrice", data=df_train)

In [ ]:
## 함수화 (코드)

def draw_categorical(target):
  data=df_train.copy()
  target = target 
  colors = ["#FF6347","#98FB98"]
  plt.figure(figsize=(10,10))
  data.groupby(target)['SalePrice'].median().plot.bar(color = colors,edgecolor = "black",linewidth = 1.5)
  plt.xlabel(target, fontsize = 15)

  plt.xticks(rotation=45)

  plt.ylabel('SalePrice',fontsize = 15)
  plt.title(target + " Vs SalePrice ",fontsize = 22)
  plt.show()

## xticks(rotation)옵션 설명

In [ ]:
## For문 모두 출력 (코드)

for target in categorical_features:
  draw_categorical(target)

## 부록 3D matplot

In [ ]:
## 이제 더 직관적인 연관관계를 파악하기 위해 3차원으로 그래프를 그려봅시다!

for feature in year_feature:
    # Year Sold를 제외한 나머지
    if feature!='YrSold':
        data=df_train.copy()
        data[feature]=data['YrSold']-data[feature]
        # 3차원 그림을 그려봅시다!
        fig = plt.figure(figsize = (12, 12))
        # 3d로 axes를 지정합니다!
        ax = plt.axes(projection ="3d")
        ax.scatter3D(data[feature],data['SalePrice'],color="tomato")
        plt.title(feature  +  " Vs SalePrice",fontsize = 22)
        plt.xlabel(feature,fontsize = 15)
        plt.ylabel('SalePrice',fontsize = 15)
        plt.grid(color="palegreen")
        plt.show()